In [13]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import networkx as nx
import music21 as msc
from funciones import *
import sys
import os
from music21 import corpus
import random 
import pyknon
from pyknon.genmidi import Midi
from pyknon.music import NoteSeq


env = msc.environment.UserSettings()
#env['musicxmlPath'] = r'C:\Program Files (x86)\Finale NotePad 2012\Finale NotePad.exe' #Path a la aplicacion Finale
#env['musicxmlPath'] = r'C:\Program Files (x86)\MuseScore 2\bin\MuseScore.exe'          #Path a la aplicacion MuseScore 
#env['musicxmlPath']='/usr/bin/mscore'
msc.environment.set('musescoreDirectPNGPath', 'C:\\Program Files (x86)\\MuseScore 2\\bin\\MuseScore.exe')
msc.environment.set('musicxmlPath', 'C:\\Program Files (x86)\\MuseScore 2\\bin\\MuseScore.exe')

#---------------------------------------------------------------------------------------------------------
#            FUNCIONES PARA ANALISIS DE MUSICA:
#---------------------------------------------------------------------------------------------------------
#Lista de funciones:
# f_xml2graph (cancion, nombre_parte=0,modelo='melodia')
# graficar (G, color_map='rainbow',layout='espiral', labels=False)
# ql_2_fig (ql)
# f_motifs_rhytmic (cancion,length,nombre_parte=0)
# f_motifs_tonal (cancion,length,nombre_parte=0)
# f_grado_dist_M (G)
# f_grado_dist_R (G)
# f_tabla (G,nombre)
# f_xml2graph_armonia (cancion, index)
# f_armon (cancion, indexes)
# graficar_armonias_undirected(G, color_map='rainbow',layout='espiral')
# graficar_armonias_directed(Armonias)
# f_dist_escalas (cancion, nombre_parte=0)
# f_full_graph(path)
# f_hierarchy(G)
# f_rewiring_directed(G)
# transitivity_motifs(G)

#-------------------

In [14]:
musicos_1 = ['bach']
#creo una lista que contenga los grafos con cada artista, va a ser una ls de ls de 5 elemen: [[M,A,R,D,U]]
H = [] # Lista para los clasicos
K = [] # Lista para los populares
G = [] # Lista completa de cada artista + 5 grafos (melodia, absoluta (dist a tonica), ritmo, arm dirigida, arm no dirigida)

for l, musico in enumerate(musicos_1):
    # Para cada musico clasico, agrega a la lista H los 4 grafos completos de cada obra
    m = nx.DiGraph()
    a = nx.DiGraph()
    r = nx.DiGraph()
    ad= nx.MultiDiGraph()
    au= nx.MultiGraph()
    paths = corpus.getComposer(musico)
    paths = paths[0:20] #corta hasta los primeros 20 temas
    for i, path in enumerate(paths):
        # Crea los cuatro grafos para cada cancion y los agrega a H como un único elemento
        M1,A1,R1,D1,U1 = f_full_graph(path,True)
        H.append([M1,A1,R1,D1,U1])
        
    for l in range(len(H)):
        # Recorre cada elemento de H (listas de cuatro grafos) y une los correspondientes
        m = f_compose(m,H[l][0])
        a = f_compose(a,H[l][1])
        r = f_compose(r,H[l][2])
        #ad= f_compose(ad,H[l][3],'AD')
        #au= f_compose(au,H[l][4],'AU')
    G.append([musico,m,a,r,ad,au])
    # Agrega a G la lista de artista + 4 grafos completos por artista

# Ahora buscamos los artistas que nosotros encontramos, vamos a trabajar con los beatles
musicos_2 = ['Beatles','Charly'] #aca se podrian ir agregando si trabajasemos con más tipo Queen y Charly
#musicos_2 = ['Charly']
# REVISAR BIEN SI LAS VOCES "SIRVEN"

for l, musico in enumerate(musicos_2):
    print(musico)
    m = nx.DiGraph()
    a = nx.DiGraph()
    r = nx.DiGraph()
    ad= nx.MultiDiGraph()
    au= nx.MultiGraph()
    myFolder = (os.getcwd()+'\\partituras\\Rock\\'+musico) # busca en el directorio actual
    arch = os.listdir(myFolder) #lista con los nombres de las canciones
    paths = [] #lista con las direcciones completas
    for i, arc in enumerate(arch):
        paths.append(myFolder+'\\'+arc)

    paths = paths[0:20]
    for i, path in enumerate(paths):
        M2,A2,R2,D2,U2 = f_full_graph(path,True)
        K.append([M2,A2,R2,D2,U2])
        
    for l in range(len(K)):
        m = f_compose(m,K[l][0])
        a = f_compose(a,K[l][1])
        r = f_compose(r,K[l][2])
        #ad= f_compose(ad,K[l][3],'AD')
        #au= f_compose(au,K[l][4],'AU')
    G.append([musico,m,a,r,ad,au]) 

musicos = musicos_1 + musicos_2

Beatles
Charly


In [15]:
#Hago las caminatas sobre las melodias
for i, musico in enumerate(musicos):
    ls=[]
    ls=random_walk_1_M(G[i][1],120)
    ls_pyk= " ".join(str(x) for x in ls)
    #con ritmos de los nodos
    #f_list2seq(ls,musico+'_ritmo_nodos_M')
    #con ritmo fijo
    notes1 = NoteSeq(ls_pyk)
    midi = Midi(1, tempo=90)
    midi.seq_notes(notes1, track=0)
    midi.write(musico+"_ritmo_fijo_M.mid")
    


In [24]:
#Hago las caminatas sobre los ritmos
for i, musico in enumerate(musicos):
    ls=[]
    ls=random_walk_1_M(G[i][3],120)
    #ls_pyk= " ".join(str(x) for x in ls)
    #con ritmos de los nodos
    f_list2seq(ls,musico+'_ritmo_nodos_R')
    #con ritmo fijo
    #notes1 = NoteSeq(ls_pyk)
    #midi = Midi(1, tempo=90)
    #midi.seq_notes(notes1, track=0)
    #midi.write(musico+"_ritmo_fijo_R.mid")
    

[0.0, 0.0, 0.017287234042553192, 0.043882978723404256, 0.0, 0.0, 0.009308510638297872, 0.0013297872340425532, 0.010638297872340425, 0.8045212765957447, 0.11303191489361702]
[0.0, 0.0, 0.0, 0.0022675736961451248, 0.05442176870748299, 0.0, 0.0, 0.06349206349206349, 0.0, 0.0, 0.013605442176870748, 0.0, 0.0, 0.009070294784580499, 0.0022675736961451248, 0.0, 0.07482993197278912, 0.31519274376417233, 0.015873015873015872, 0.0022675736961451248, 0.022675736961451247, 0.3129251700680272, 0.0022675736961451248, 0.034013605442176874, 0.015873015873015872, 0.05895691609977324, 0.0]


PitchException: Cannot make a step out of 'R'

In [17]:
notes1

<Seq: [<D>, <A>, <F>, <R: 0.2>, <A>, <G>, <E>, <G>, <R: 0.25>, <C>, <R: 0.25>, <R: 0.25>, <R: 0.25>, <G>, <G>, <A>, <D>, <C>, <C>, <D>, <R: 0.2>, <R: 0.2>, <C>, <R: 0.25>, <D>, <A>, <B>, <D>, <A>, <D>, <E>, <F#>, <E>, <F#>, <G>, <C>, <R: 0.25>, <R: 0.25>, <R: 0.25>, <G>, <F#>, <E>, <E>, <E>, <D>, <D>, <C>, <C>, <C>, <C>, <C>, <C>, <C>, <C>, <C>, <R: 0.25>, <C>, <C>, <C>, <R: 0.25>, <G>, <G>, <G>, <F>, <F>, <A>, <C>, <R: 0.25>, <R: 0.25>, <B>, <G>, <B>, <A>, <A>, <G>, <G>, <A>, <G>, <F>, <G>, <F>, <G>, <F#>, <D#>, <E>, <G>, <B>, <D>, <C>, <A>, <A>, <C#>, <B>, <B>, <D>, <F>, <F>, <E>, <D>, <E>, <E>, <D>, <D>, <D>, <D>, <A>, <G>, <B>, <F#>, <F#>, <D#>, <R: 0.2>, <R: 0.2>, <C#>, <B>, <B>, <C>, <D>, <C>, <R: 0.2>, <R: 0.2>]>

In [22]:
#toma un grafo y la cantidad de los nuevos nodos. Devuelve una lista random con las notas
def random_walk_1_M(G,k):
    #creo un dict que tiene la voz como .key y la lista de notas como .value
    ls = []
    nodos=list(G.nodes())
    #Me armo un dict, con el key del nombre del nodo y una lista
    weights={} 
    
    M=nx.to_numpy_matrix(G) #obtenemos la matriz de adyacencia no esparsa
    L=M.shape[1]
    #normalizamos
    for i, nodo in enumerate(nodos):
        if (np.sum(M[i]) > 0):
            M[i] = M[i]/np.sum(M[i])
        #los pesos de salto son los valores de las filas normalizadas.
        #paso M[i] a una lista, le hago .tolist()[0] asi no me grafica extra corchetes
        weights.update({str(nodo): M[i].tolist()[0]})  
        
    
    #Hago la caminata
    #Me paro en algun nodo inicial random
    nodo_i=random.choice(nodos)
    print(weights[nodo_i])
    for i in range(k):
        if i == 0: #para el primer paso
            ls.append(nodo_i)
            #Para ese nodo inicial veo que elementro de matriz le corresponde
            #se podria hacer dos random walks, una con los pesos acumulados (cum_weights) y otras sin.
            #la lista tiene que ser los nodos 
            nodo_ran=random.choices(nodos, weights[nodo_i],k=1)#estoy teniendo problemas con este m
            #weights[str(nodo_i)] me da lista de primeros vecinos
            ls.append(nodo_ran[0]) #agrego el que haya salido. Le pongo [0] xq random.choices me devuelve una lista
        else:
            #veo los vecinos del nuevo nodo
            nodo_ran=random.choices(nodos, weights[nodo_ran[0]],k=1)
            ls.append(nodo_ran[0]) #agrego el que haya salido
        
    return(ls)

In [ ]:
#toma un grafo 3D y cantidad de voces. Para cada voz va a hacer una random walk por #cada uno de dichos nodos
#devuelve 'voces' cantidad de listas de los nodos que fue tomando
def f_rndm_walk(G, voces, k):
    #creo un dict que tiene la voz como .key y la lista de notas como .value
    ls = {}
    enlaces=[]
    nodos=[]
    for i,voz in enumerate(voces):
        ls[str(voz)] = []

    #tendriamos que hacerlo empezando de la voz principal 
    #y cada vez q  toque algun nodo que tiene un enlace no dirigido abrir una rama para ese layer
    for i, layer in enumerate(G):
        enlaces.append(layer.edges()) 
        nodos.append(layer.nodes())
    
    
    for j, layer in enumerate(G):
        M = nx.adjacency_matrix(layer) #obtenemos la matriz de adyacencia
        #normalizamos
        for i in range(M.shape[1]):
            if (np.sum(M[i]) > 0):
                M[i] = M[i]/np.sum(M[i])
        restart_prob=M.min()

        #weights=[] hay que averiguar sobre como son las matrices de adj para multigraphs y layers


        for i, nodo in enumerate(nodos[j]):
            p=random.random()
        #Aca revisamos si todos los enlaces son dirigidos que siga el algoritmo. Si no que agregue otra caminata de borracho
        #para el otro layer
            if len(layer.in_edges(nodo))>0 or len(layer.out_edges(nodo))>0 and len(layer.edges(nodo))==0:
                if p < restart_prob:
                    #do restart
                    break

                else:
                    #choose next node    
                    #se podria hacer dos random walks, una con los pesos acumulados (cum_weights) y otras sin. 
                    nodo_ran=random.choices(nodos[j], weights=[], cum_weights=None, k=1)
                    ls[str(layer)].append(nodo_ran)


            if len(G.edges(nodo))>0: #si se encuentra con un no dirigido 
                if p < restart_prob:
                    #do restart
                    break
                else:
                    #toma el nodo del otro layer al que está conectado y empieza una random walk para ese layer   
                    ls[str(layer)].append(nodo)
                    for i, nodo in enumerate(layer):
                    #Aca revisamos si todos los enlaces son dirigidos que siga el algoritmo. Si no que agregue otra caminata de borracho
                    #para el otro layer
                        if len(layer.in_edges(nodo))>0 or len(layer.in_edges(nodo))>0 and len(layer.edges(nodo))==0:
                            if p < restart_prob:
                                #do restart

                            else:
                                #choose next node    
                                #se podria hacer dos random walks, una con los pesos acumulados (cum_weights) y otras sin. 
                                nodo_ran=random.choices(nodos, weights=[], cum_weights=None, k=1)
                                ls[str(voz)].append(nodo_ran)

    return(ls)
